In [1]:
pip install --upgrade category_encoders

Requirement already up-to-date: category_encoders in /Users/Inceptive/.local/share/virtualenvs/amazon-ratings-dash-app-qCkU82-A/lib/python3.7/site-packages (2.0.0)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install plotly 

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Various Imports

import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error
import plotly.express as px

In [6]:
# bringing in THE dataset

df = pd.read_csv('https://query.data.world/s/dh4qojydat5rryqzot3q4x3osoj6dc')
df.head()

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1.0,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ..."
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,NaN,2,1.0,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ..."
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2.0,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel..."
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,NaN,1,2.0,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,NaN,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,I love it // 5.0 // 22 July 2013 // By\n \n...,NaN
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,£32.19,NaN,3,2.0,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,Birthday present // 5.0 // 14 April 2014 // By...,NaN


In [7]:
# Baseline
df['average_review_rating'].value_counts(normalize=True)

5.0 out of 5 stars    0.514927
4.0 out of 5 stars    0.131437
4.5 out of 5 stars    0.071929
4.8 out of 5 stars    0.056802
4.7 out of 5 stars    0.052895
4.3 out of 5 stars    0.040773
4.6 out of 5 stars    0.039271
4.4 out of 5 stars    0.029854
4.9 out of 5 stars    0.023743
4.2 out of 5 stars    0.021739
4.1 out of 5 stars    0.014526
3.9 out of 5 stars    0.000701
3.6 out of 5 stars    0.000301
3.0 out of 5 stars    0.000301
3.3 out of 5 stars    0.000200
3.7 out of 5 stars    0.000200
3.5 out of 5 stars    0.000200
2.3 out of 5 stars    0.000100
3.8 out of 5 stars    0.000100
Name: average_review_rating, dtype: float64

In [8]:
# Some Wrangling and Cleaning, still much more to do though

# removing the Euros symbol to later convert it to floats
df['price'] = df['price'].str[1:]

# removing the string after the number of availalble stock to convert to an interger later
df['number_available_in_stock'] = df['number_available_in_stock'].str.extract('(\d+)')
df['number_available_in_stock'] = df['number_available_in_stock'].fillna(6).astype(int)

# removing the unneccessary clutter around the rating so that I can later convert to float
df['average_review_rating'] = df['average_review_rating'].str.extract('(\d+(?:\.\d+)?)')

# removing the URL to the new product, will continue to clean and get it just down to the item
df['customers_who_bought_this_item_also_bought'] = df['customers_who_bought_this_item_also_bought'].str[24:]
df['items_customers_buy_after_viewing_this_item'] = df['items_customers_buy_after_viewing_this_item'].str[24:]

# rather than have category>subcategory>subsubcategory in one column, I gave each their own column
df['category'] = df['amazon_category_and_sub_category'].str.split('>').str[0].tolist()
df['sub_category'] = df['amazon_category_and_sub_category'].str.split('>').str[1].tolist()
df['sub_sub_category'] = df['amazon_category_and_sub_category'].str.split('>').str[2].tolist()
df['sub_sub_sub_category'] = df['amazon_category_and_sub_category'].str.split('>').str[3].tolist()

# fixing commas in strings in number of reviews and then converting column to floats
df['number_of_reviews'] = df['number_of_reviews'].replace('1,040','1040')
df['number_of_reviews'] = df['number_of_reviews'].replace('1,399','1399')
df['number_of_reviews'] = df['number_of_reviews'].fillna(0.0).astype(int)

# converting Average review rating to floats 
df['average_review_rating'] = df['average_review_rating'].astype(float)
#df['average_review_rating'] = df['average_review_rating'].fillna(4.5, inplace=False)


# fixing strings in price column so they can be converted to floats
df['price'] = df['price'].str.split(' - £').str[0].tolist()
df['upper_range_price'] = df['price'].str.split(' - £').str[1].tolist()
df['price'] = df['price'].replace('2,439.92', '2439.92')
df['price'] = df['price'].astype(float)

# changing nans in the column to the mean of the column
df['price'] = df['price'].fillna(20.2781097489784, inplace=False)
df['number_of_answered_questions'] = df['number_of_answered_questions'].fillna(2, inplace=False)

# Changing Description to 1 or 0 for if one is present
df['description'] = df['description'].fillna(value=0)
df['new_description'] = [0 if x is 0 else 1 for x in df['description']]

# Changing customer Reviews to 1 or 0 for if one is present
df['customer_reviews'] = df['customer_reviews'].fillna(value=0)
df['new_customer_reviews'] = [0 if x is 0 else 1 for x in df['customer_reviews']]

# Changing customer Q&A to 1 or 0 for if one is present
df['customer_questions_and_answers'] = df['customer_questions_and_answers'].fillna(value=0)
df['new_customer_questions_and_answers'] = [0 if x is 0 else 1 for x in df['customer_questions_and_answers']]

# Changing  product info to 1 or 0 for if one is present
df['product_information'] = df['product_information'].fillna(value=0)
df['new_product_information'] = [0 if x is 0 else 1 for x in df['product_information']]

# Changing Product Description to 1 or 0 for if one is present
df['product_description'] = df['product_description'].fillna(value=0)
#df['new_product_description'] = [0 if x is 0 else 1 for x in df['product_description']]

# unusable variance 
unusable_variance = ['uniq_id',
                     'amazon_category_and_sub_category', 
                     'customers_who_bought_this_item_also_bought', 
                     'description',
                     'product_information', 
                     'product_description', 
                     'items_customers_buy_after_viewing_this_item', 
                     'customer_questions_and_answers',
                     'customer_reviews', 
                     'sellers', 
                     'upper_range_price',]
df = df.drop(columns=unusable_variance)


# dropping some categoricals for the sake of a linear model, will reuse these at later point
unusable_categoricals = ['product_name',
                        # 'manufacturer', 
                         #'category',
                         #'sub_category', 
                         'sub_sub_category', 
                         'sub_sub_sub_category'
                        ]

df = df.drop(columns=unusable_categoricals)


In [9]:
df = df.dropna()
print(df.shape)
df.head()

(9982, 9)


,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,new_description,new_customer_reviews,new_customer_questions_and_answers,new_product_information
0,3.42,5,15,1.0,4.9,1,1,1,1
1,16.99,6,2,1.0,4.5,1,1,1,1
2,9.99,2,17,2.0,3.9,1,1,1,1
3,39.99,6,1,2.0,5.0,1,1,0,1
4,32.19,6,3,2.0,4.7,1,1,0,1


In [10]:
df.dtypes

price                                 float64
number_available_in_stock               int64
number_of_reviews                       int64
number_of_answered_questions          float64
average_review_rating                 float64
new_description                         int64
new_customer_reviews                    int64
new_customer_questions_and_answers      int64
new_product_information                 int64
dtype: object

In [11]:
# print the shape and checking nulls
print(df.shape)
df.isnull().sum(axis=0)

(9982, 9)


price                                 0
number_available_in_stock             0
number_of_reviews                     0
number_of_answered_questions          0
average_review_rating                 0
new_description                       0
new_customer_reviews                  0
new_customer_questions_and_answers    0
new_product_information               0
dtype: int64

In [12]:
# now making train/val/test split



#train, val = train_test_split(df, train_size=.8, test_size = .2, 
 #                             random_state=69)

In [13]:
# Assigning X feature matrices and y target vectors

# assigning the target
target = 'average_review_rating'

X_train = df.drop(columns=target)
y_train = df[target]

#X_val = val.drop(columns=target)
#y_val = val[target]

In [14]:
pipeline = make_pipeline(
        ce.OneHotEncoder(),
        SimpleImputer(),
        StandardScaler(),
        LinearRegression()

)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_train)
print('Mean Absolute Error: ', mean_absolute_error(y_train, y_pred))

[12:10:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/Inceptive/.local/share/virtualenvs/amazon-ratings-dash-app-qCkU82-A/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/Inceptive/.local/share/virtualenvs/amazon-ratings-dash-app-qCkU82-A/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning:

Series.base is deprecated and will be removed in a future version



Mean Absolute Error:  0.27910598967171363


In [15]:
def predict(price, number_available_in_stock, number_of_reviews, number_of_answered_questions,
           new_description, new_customer_reviews, new_customer_questions_and_answers,
           new_product_information):
 
  
    y_pred = pipeline.predict([[price, number_available_in_stock, number_of_reviews, number_of_answered_questions,
           new_description, new_customer_reviews, new_customer_questions_and_answers,
           new_product_information]])
  
    estimate = y_pred[0]
  
  
    return estimate

predict(16.99, 3, 20, 1.0, 1,0,1,1)

4.5224605

In [16]:
from ipywidgets import interact 
interact(predict, price = (0,250), 
         number_available_in_stock = (0,95),
         number_of_reviews = (0,1400), 
         number_of_answered_questions = (0,40),
         new_description = (0,1), 
         new_customer_reviews = (0,1),
         new_customer_questions_and_answers = (0,1),
         new_product_information = (0,1))

interactive(children=(IntSlider(value=125, description='price', max=250), IntSlider(value=47, description='num…

<function __main__.predict(price, number_available_in_stock, number_of_reviews, number_of_answered_questions, new_description, new_customer_reviews, new_customer_questions_and_answers, new_product_information)>

In [17]:
from joblib import dump
dump(pipeline, 'pipeline.joblib') 

['pipeline.joblib']